In [1]:
import numpy as np
import pandas as pd
import torch
from load_msi import create_msi
from visualization import (
        generate_nx_graph, generate_khop_subgraph, generate_pyvis_graph, 
        visualize_tsne_embeddings, visualize_pca_embeddings)
from training import split_data, create_minibatches, train_link_predictor
from torch_geometric.utils import to_networkx, degree
from torch_geometric.nn import to_hetero

from gnn import GAT, GIN, DotProductLinkPredictor
import torch_geometric.transforms as T
print(torch.__version__)

1.13.1+cu117


In [2]:
msi = create_msi("msi", 
                 num_features = 1,
                 data_dir="../multiscale_interactome/data")

data = msi.hetero_data
data = T.ToUndirected()(data)

In [3]:
# Generate and visualize graphs (HIV: id = C0019693, gid = 840)
##G, G_undirected = generate_nx_graph(msi.data)
##subG = generate_khop_subgraph(msi, G, 840, 1)
##py_graph = generate_pyvis_graph(subG)
##py_graph.show("test.html")

In [4]:
# Set the device.
device = torch.device('cuda' if torch.cuda.is_available() else 'cpu')
print(f"Device: {device}")

Device: cuda


In [5]:
edge_type = ("dz", "implicates", "prot")
id_map = msi.mappings["dz_id_gid_map"]
name_map = msi.mappings["dz_id_name_map"]



degrees = degree(data[edge_type].edge_index[0]).numpy()
sorted_gids = np.argsort(degrees)[::-1]
sorted_degs = np.sort(degrees)[::-1]


#for gid, deg in zip(sorted_gids, sorted_degs):
#    id = list(id_map.keys())[list(id_map.values()).index(gid)]
#    name = name_map[id]
#    print(f"Disease: {name:<45}   |   1-hop neighbors: {deg:>5}   |   id: {id}")


In [6]:
# Prepare data for GNN.
edge_types = ("drug", "treats", "dz")
rev_edge_types = ("dz", "rev_treats", "drug")

# Split the data into train, validation, and test sets.
train_data, val_data, test_data = split_data(
        data, val_size=0.1, test_size=0.1, supervision_ratio=0.3,
        edge_types=edge_types, neg_sampling_ratio=1.0,
        rev_edge_types=rev_edge_types)


# Create minibatches.
train_loader = create_minibatches(
        train_data, edge_types=edge_types, num_khop_neighbors=[50,25,10], neg_sampling_ratio=1.0, batch_size=16)
val_loader = create_minibatches(
        val_data, edge_types=edge_types, num_khop_neighbors = [50,25,10], batch_size=16)



In [7]:
# Instantiate the model and generate some initial node embeddings prior to training.
model = GAT(4, 64, 16)
model = to_hetero(model, data.metadata(), aggr='sum').to(device)
predictor = DotProductLinkPredictor(32, 32, 1).to(device)

In [8]:
optimizer = torch.optim.Adam(model.parameters(), lr=0.001, weight_decay=5e-4)
#model.reset_parameters()
#predictor.reset_parameters()

# Train and validate the model for a given number of epochs.
for epoch in range(1, 50+1):
    
    # Training..
    loss, auc = train_link_predictor(
            model, predictor, "drug", "dz", edge_types=edge_types, 
            loader=train_loader, optimizer=optimizer, device=device, train=True)
    
    # Validation.
    val_loss, val_auc = train_link_predictor(
            model, predictor, "drug", "dz", edge_types=edge_types, 
            loader=val_loader,  optimizer=optimizer, device=device, train=False)
    
    print(f"Epoch     : {epoch}")
    print(f" Loss     : {loss:<10.4f} |     AUC:     {auc:>10.4f}")
    print(f" Val_Loss : {val_loss:<10.4f} |     Val_AUC: {val_auc:>10.4f}")

100%|██████████| 74/74 [00:10<00:00,  7.36it/s]


Epoch     : 1
 Loss     : 38.7276    |     AUC:         0.5138
 Val_Loss : 0.9099     |     Val_AUC:     0.4989


100%|██████████| 74/74 [00:10<00:00,  7.32it/s]


Epoch     : 2
 Loss     : 0.8807     |     AUC:         0.5207
 Val_Loss : 0.8202     |     Val_AUC:     0.5069


100%|██████████| 74/74 [00:10<00:00,  7.20it/s]


...

Epoch     : 48
 Loss     : 0.3192     |     AUC:         0.9156
 Val_Loss : 0.2240     |     Val_AUC:     0.9647


100%|██████████| 74/74 [00:10<00:00,  7.17it/s]


Epoch     : 49
 Loss     : 0.3385     |     AUC:         0.9021
 Val_Loss : 0.2565     |     Val_AUC:     0.9599


100%|██████████| 74/74 [00:10<00:00,  7.13it/s]

Epoch     : 50
 Loss     : 0.3046     |     AUC:         0.9172
 Val_Loss : 0.2780     |     Val_AUC:     0.9456


In [16]:

msi = create_msi("msi-hiv", 
                 num_features = 1,
                 data_dir="../multiscale_interactome/data", 
                 extra_dz_data="../data/hiv/hiv_protein30.tsv")

hiv_data = msi.hetero_data
hiv_data = T.ToUndirected()(hiv_data)
#(HIV: id = C0019693, gid = 840)
hiv_id = "C0019693"
hiv_gid = 840



In [17]:
def get_node_embeddings(model, data):
    model.eval()
    z = model(data.x_dict, data.edge_index_dict)
    return z

In [18]:
hiv_data.to("cpu")
model.to("cpu")
predictor.to("cpu")

DotProductLinkPredictor(
  (lin1): Linear(32, 32, bias=True)
  (lin2): Linear(32, 1, bias=True)
)

In [19]:
torch.cuda.empty_cache()
#print("torch.cuda.memory_allocated: %fGB"%(torch.cuda.memory_allocated(0)/1024/1024/1024))
#print("torch.cuda.memory_reserved: %fGB"%(torch.cuda.memory_reserved(0)/1024/1024/1024))
#print("torch.cuda.max_memory_reserved: %fGB"%(torch.cuda.max_memory_reserved(0)/1024/1024/1024))
#print(torch.cuda.memory_summary(device=None, abbreviated=False))

In [26]:
z = get_node_embeddings(model, hiv_data)
z = model(hiv_data.x_dict, hiv_data.edge_index_dict)
z_drugs = z["drug"]
z_dz = z["dz"]
hiv_emb = z["dz"][hiv_gid]

In [27]:
from numpy import dot
from numpy.linalg import norm



#hiv_emb = hiv_emb.cpu().detach().numpy()
drug_edge_preds = []
dz_edge_preds = []
for drug_emb in z_drugs:
    drug_edge_pred = predictor(drug_emb, hiv_emb)
    drug_edge_pred = drug_edge_pred.detach().numpy()
    drug_edge_preds.append(drug_edge_pred)
drug_sorted_edges = np.sort(drug_edge_preds)[::-1]
drug_sorted_ind = np.argsort(drug_edge_preds)[::-1]


for dz_emb in z_dz:
    dz_edge_pred = predictor(dz_emb, hiv_emb)
    dz_edge_pred = dz_edge_pred.detach().numpy()
    dz_edge_preds.append(dz_edge_pred)
dz_sorted_edges = np.sort(dz_edge_preds)[::-1]
dz_sorted_ind = np.argsort(dz_edge_preds)[::-1]




In [28]:
import pandas as pd
id_list = []
drug_list = []
sim_list = []
for ind, edge in zip(drug_sorted_ind, drug_sorted_edges):
    id = [i for i in msi.mappings["drug_id_gid_map"] if msi.mappings["drug_id_gid_map"][i] == ind][0]
    drug = msi.mappings["drug_id_name_map"][id]
    id_list.append(id)
    drug_list.append(drug)
    sim_list.append(edge)
    print(f"Drug: {drug:<30}    |    HIV edge pred: {edge:>10.4}")

res_dict = {"id":id_list, "name" : drug_list, "similarity" : sim_list}
res_df = pd.DataFrame(res_dict)

print("\n\n\n")

for ind, edge in zip(dz_sorted_ind, dz_sorted_edges):
    id = [i for i in msi.mappings["dz_id_gid_map"] if msi.mappings["dz_id_gid_map"][i] == ind][0]
    dz = msi.mappings["dz_id_name_map"][id]
    print(f"DZ: {dz:<30}    |    HIV edge pred: {edge:>10.4}")

res_dict = {"id":id_list, "name" : drug_list, "similarity" : sim_list}
res_df = pd.DataFrame(res_dict)


Drug: nan                               |    HIV edge pred:      2.233
Drug: Dexfenfluramine                   |    HIV edge pred:      2.072
Drug: nan                               |    HIV edge pred:      2.054
Drug: enoximone                         |    HIV edge pred:      1.985
Drug: nan                               |    HIV edge pred:      1.898
Drug: mianserin                         |    HIV edge pred:      1.873
Drug: viloxazine                        |    HIV edge pred:      1.856
Drug: risperidone                       |    HIV edge pred:      1.832
Drug: hyaluronidase (ovine)             |    HIV edge pred:      1.821
Drug: ethynodiol-diacetate              |    HIV edge pred:      1.815
Drug: Azelaic Acid                      |    HIV edge pred:      1.814
Drug: tolbutamide                       |    HIV edge pred:      1.812
Drug: carprofen                         |    HIV edge pred:      1.797
Drug: mitiglinide                       |    HIV edge pred:      1.762
Drug: 

In [ ]:
# Write results to file
#res_df.to_csv("../results/res1_loss0.47_auc0.76.tsv", sep="\t", index=False)